<a href="https://colab.research.google.com/github/aniaprokosheva/urban_kgiop_objects/blob/main/urban_kgiop_objects_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект: Улицы. Объекты культурного наследия.

**Цель исследования:** отобразить объкты культурного наследия для улиц в выбранном районе Санкт-Петербурга.
* KGIOP - Комитет по государственному контролю, использованию и охране памятников истории и культуры

**Библиотеки:** pandas, geopandas, osmnx

**Содержание:**
1. Общая информация (загрузка модулей и библиотек, константы и функции)
2. Территория Санкт-Петербурга
3. Улицы, которые входят в выбранный район.
4. Объекты культурного наследия выбранной территории.
    4.3. Количество объектов, попадающие в заданный буфер вокруг улицы.
5. Выводы
6. Сохранение файлов

## 1 Общая информация

In [1]:
#дополнительные модули
!pip install folium -U
!pip install geopandas mapclassify osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 4.1 MB/s eta 0:00:00


In [2]:
#установка библиотек
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [3]:
#все константs, использованные в проекте
TILES = "CartoDB positron" #подложка карт

TERRITORY_NAME = 'Красногвардейский район, Санкт-Петербург' #выбранный район
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing" #слой с улицами, которые входят в выбранный район
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/python-urban/main/kgiop_objects.geojson" #слой с объектами культурного наследия

In [4]:
# функция, позволяющая читать ссылку для скачивания с google drive
def get_google_drive_download_url(url: str) -> str:
    """
    Функция возвращает ссылку для скачивания с google drive.
    """
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

## 2 Территория

### 2.1 Загрузка территории из OSM (Extract)

In [5]:
#подгружаем территорию с OSM и получаем GeoDataFrame с помощью `ox.geocode_to_gdf()
territory = ox.geocode_to_gdf(TERRITORY_NAME)
territory.explore(tiles=TILES)

## 3 Улицы

### 3.1 Загрузка файла с улицами из google drive

In [6]:
url = get_google_drive_download_url(STREETS_FILE_URL)
gdf_streets = gpd.read_file(url, mask=territory)  # фильтр улиц по маске геометрии территории, полученной ранее
gdf_streets

,type,id,tags,geometry
0,way,4078548,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3385685.754 8387961.891, 3385692.2..."
1,way,4454334,"{'highway': 'tertiary', 'junction': 'roundabou...","LINESTRING (3388641.888 8396122.519, 3388624.9..."
2,way,4454660,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3388144.278 8393976.488, 3388156.8..."
3,way,4454692,"{'highway': 'unclassified', 'lanes': '2', 'lit...","LINESTRING (3386659.454 8393215.475, 3386629.8..."
4,way,4455562,"{'highway': 'secondary', 'lanes': '2', 'layer'...","LINESTRING (3390821.701 8403547.048, 3390778.0..."
...,...,...,...,...
6316,way,981878572,{'highway': 'service'},"LINESTRING (3390484.437 8390881.717, 3390544.7..."
6317,way,982791402,"{'highway': 'service', 'surface': 'asphalt'}","LINESTRING (3390016.116 8405605.740, 3390030.0..."
6318,way,982791403,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3390016.116 8405605.740, 3390104.3..."
6319,way,982791404,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3389902.759 8405833.105, 3390016.1..."


### 3.2 Обработка данных с улицами

In [7]:
tags = pd.json_normalize(gdf_streets["tags"]) # json_normalize позволяет преобразовать вложенные структуры данных в json файле в таблицу
tags

,foot,highway,lanes,lit,maxspeed,name,oneway,postal_code,surface,junction,...,parking:lane:right:time_interval,motor_vehicle,placement,busway,cycleway:right,bus,bicycle,horse,footway,noexit
0,no,residential,2,yes,RU:urban,Шепетовская улица,yes,195027,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,tertiary,1,yes,RU:urban,Екатерининский проспект,yes,NaN,asphalt,roundabout,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no,primary,3,yes,RU:urban,проспект Энергетиков,yes,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,unclassified,2,no,RU:urban,NaN,NaN,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,secondary,2,yes,RU:rural,Пискарёвский проспект,yes,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6316,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6317,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6318,NaN,residential,1,yes,20,проспект Маршака,yes,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6319,NaN,residential,1,yes,40,проспект Маршака,yes,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 94 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   foot                              846 non-null    object
 1   highway                           6321 non-null   object
 2   lanes                             1432 non-null   object
 3   lit                               1496 non-null   object
 4   maxspeed                          1468 non-null   object
 5   name                              1340 non-null   object
 6   oneway                            1251 non-null   object
 7   postal_code                       119 non-null    object
 8   surface                           2371 non-null   object
 9   junction                          11 non-null     object
 10  trolley_wire                      317 non-null    object
 11  layer                             106 non-null    object
 12  smoothness          

In [9]:
extract_tags = ['name']  # добавляем "name"

gdf_streets = gdf_streets.join(tags[extract_tags])
gdf_streets

,type,id,tags,geometry,name
0,way,4078548,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3385685.754 8387961.891, 3385692.2...",Шепетовская улица
1,way,4454334,"{'highway': 'tertiary', 'junction': 'roundabou...","LINESTRING (3388641.888 8396122.519, 3388624.9...",Екатерининский проспект
2,way,4454660,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3388144.278 8393976.488, 3388156.8...",проспект Энергетиков
3,way,4454692,"{'highway': 'unclassified', 'lanes': '2', 'lit...","LINESTRING (3386659.454 8393215.475, 3386629.8...",NaN
4,way,4455562,"{'highway': 'secondary', 'lanes': '2', 'layer'...","LINESTRING (3390821.701 8403547.048, 3390778.0...",Пискарёвский проспект
...,...,...,...,...,...
6316,way,981878572,{'highway': 'service'},"LINESTRING (3390484.437 8390881.717, 3390544.7...",NaN
6317,way,982791402,"{'highway': 'service', 'surface': 'asphalt'}","LINESTRING (3390016.116 8405605.740, 3390030.0...",NaN
6318,way,982791403,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3390016.116 8405605.740, 3390104.3...",проспект Маршака
6319,way,982791404,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3389902.759 8405833.105, 3390016.1...",проспект Маршака


In [10]:
gdf_streets.dropna(
    how="all",  # удалить если отсутствуют значения во всех указанных столбцах
    subset=["name"],  # столбцы, в которых надо искать пропуски
    inplace=True
)
gdf_streets

,type,id,tags,geometry,name
0,way,4078548,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3385685.754 8387961.891, 3385692.2...",Шепетовская улица
1,way,4454334,"{'highway': 'tertiary', 'junction': 'roundabou...","LINESTRING (3388641.888 8396122.519, 3388624.9...",Екатерининский проспект
2,way,4454660,"{'foot': 'no', 'highway': 'primary', 'lanes': ...","LINESTRING (3388144.278 8393976.488, 3388156.8...",проспект Энергетиков
4,way,4455562,"{'highway': 'secondary', 'lanes': '2', 'layer'...","LINESTRING (3390821.701 8403547.048, 3390778.0...",Пискарёвский проспект
5,way,4455658,"{'highway': 'unclassified', 'lanes': '1', 'lit...","LINESTRING (3388626.448 8394297.512, 3388806.4...",проспект Маршала Блюхера
...,...,...,...,...,...
6313,way,979810037,"{'highway': 'tertiary', 'lanes': '2', 'lit': '...","LINESTRING (3395385.199 8385316.349, 3395302.6...",Хасанская улица
6314,way,979810038,"{'highway': 'tertiary', 'lanes': '2', 'lit': '...","LINESTRING (3395270.184 8385325.237, 3395300.5...",Хасанская улица
6318,way,982791403,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3390016.116 8405605.740, 3390104.3...",проспект Маршака
6319,way,982791404,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3389902.759 8405833.105, 3390016.1...",проспект Маршака


In [11]:
# проверка наличия дубликатов по полям "id", "geometry", "name"
gdf_streets.duplicated(subset = ["id", "geometry", "name"]).value_counts()

False    1340
dtype: int64

In [12]:
# уникальные значения
gdf_streets["name"].is_unique

False

In [13]:
#группировка геометрии по столбцу name и объединение геометрии в одну
gdf_streets = gdf_streets.dissolve(by="name")
gdf_streets

,geometry,type,id,tags
name,,,,
1-й Охтинский путепровод,"LINESTRING (3385032.231 8389412.855, 3385026.9...",way,174878843,"{'bridge': 'yes', 'highway': 'service', 'layer..."
1-я Жерновская улица,"LINESTRING (3393354.687 8391674.447, 3393424.7...",way,35552052,"{'highway': 'residential', 'lanes': '1', 'lit'..."
1-я Поперечная улица,"LINESTRING (3400575.014 8394720.248, 3400536.9...",way,25762136,"{'addr:suburb': 'Ново-Ковалёво', 'highway': 'r..."
1-я линия,"LINESTRING (3400408.291 8394100.825, 3400263.2...",way,222472806,"{'addr:suburb': 'Ново-Ковалёво', 'highway': 'r..."
2-й Охтинский путепровод,"LINESTRING (3385001.317 8388918.283, 3384996.1...",way,174878847,"{'bridge': 'yes', 'highway': 'service', 'layer..."
...,...,...,...,...
улица Стасовой,"MULTILINESTRING ((3387289.000 8392529.340, 338...",way,4459864,"{'highway': 'residential', 'lanes': '1', 'lit'..."
улица Стахановцев,"MULTILINESTRING ((3384905.081 8383712.036, 338...",way,31366301,"{'foot': 'no', 'highway': 'tertiary', 'lanes':..."
улица Химиков,"MULTILINESTRING ((3391832.939 8392062.738, 339...",way,31380613,"{'foot': 'no', 'highway': 'tertiary', 'lanes':..."


In [14]:
# переименуем индекс
gdf_streets.index.rename("Название улицы", inplace=True)
gdf_streets.head()

,geometry,type,id,tags
Название улицы,,,,
1-й Охтинский путепровод,"LINESTRING (3385032.231 8389412.855, 3385026.9...",way,174878843,"{'bridge': 'yes', 'highway': 'service', 'layer..."
1-я Жерновская улица,"LINESTRING (3393354.687 8391674.447, 3393424.7...",way,35552052,"{'highway': 'residential', 'lanes': '1', 'lit'..."
1-я Поперечная улица,"LINESTRING (3400575.014 8394720.248, 3400536.9...",way,25762136,"{'addr:suburb': 'Ново-Ковалёво', 'highway': 'r..."
1-я линия,"LINESTRING (3400408.291 8394100.825, 3400263.2...",way,222472806,"{'addr:suburb': 'Ново-Ковалёво', 'highway': 'r..."
2-й Охтинский путепровод,"LINESTRING (3385001.317 8388918.283, 3384996.1...",way,174878847,"{'bridge': 'yes', 'highway': 'service', 'layer..."


In [15]:
# удалим столбцы
drop_columns = [
    "id",
    "type",
    "tags",
]
gdf_streets.drop(columns=drop_columns, inplace=True)
gdf_streets.head()

,geometry
Название улицы,
1-й Охтинский путепровод,"LINESTRING (3385032.231 8389412.855, 3385026.9..."
1-я Жерновская улица,"LINESTRING (3393354.687 8391674.447, 3393424.7..."
1-я Поперечная улица,"LINESTRING (3400575.014 8394720.248, 3400536.9..."
1-я линия,"LINESTRING (3400408.291 8394100.825, 3400263.2..."
2-й Охтинский путепровод,"LINESTRING (3385001.317 8388918.283, 3384996.1..."


In [16]:
gdf_streets.explore(tiles=TILES)

## 4 Объекты культурного наследия

### 4.1 Загрузка объектов культурного наследия

In [17]:
gdf_kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory) # фильтр объектов по маске геометрии территории, полученной ранее
gdf_kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,2733,—,Казармы служителей Охтинского Порохового завод...,20 в. нач.,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3393917.541 8391682.542)
1,2734,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 58, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393942.031 8391697.888)
2,2735,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 64, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393893.051 8391664.084)
3,2736,—,Доходный дом П.И. Иванова,1906-1907,"Большая Пороховская ул., 16; Среднеохтинский п...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385859.012 8389439.579)
4,2737,—,Особняк П.И. Иванова,1901,"Большая Пороховская ул., 18",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385918.345 8389472.261)
...,...,...,...,...,...,...,...,...,...
113,2893,—,Памятник общественному деятелю Гроту К.К.,1906,"Шаумяна пр., 44",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3386904.302 8383501.147)
114,2894,—,Дом призрения бедных духовного звания при Боль...,—,"Шепетовская ул., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-21 от 22.06.2009,Памятник,POINT (3386353.938 8387883.003)
115,2895,—,Триумфальные пилоны в честь защитников Ленингр...,1952-1953,"Коммуны ул., д.50, сооружение 1, лит. А; д.51,...",объект культурного наследия регионального знач...,Распоряжение КГИОП от 20.04.2016 № 10-139,Памятник,POINT (3394240.145 8390795.015)
116,9659,—,Участок сохранившегося культурного слоя с сохр...,—,северная часть квартала 11 М.Охты (Новочеркасс...,выявленный объект культурного наследия,Распоряжение КГИОП № 342-р от 21.10.2020,Памятник,POINT (3384950.867 8384360.913)


### 4.2 Обработка объектов культурного наследия

In [18]:
# оставляем только столбцы object_name, object_location, object_type,
drop_columns = [
    "id",
    "ensemble_name",
    "occurrence_time",
    "historical_category",
    "normative_act"
]
gdf_kgiop_objects.drop(columns=drop_columns, inplace=True)
gdf_kgiop_objects.head()

,object_name,object_location,object_type,geometry
0,Казармы служителей Охтинского Порохового завод...,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",Ансамбль,POINT (3393917.541 8391682.542)
1,Казарма,"2-я Жерновская ул., 58, лит. А",Памятник,POINT (3393942.031 8391697.888)
2,Казарма,"2-я Жерновская ул., 64, лит. А",Памятник,POINT (3393893.051 8391664.084)
3,Доходный дом П.И. Иванова,"Большая Пороховская ул., 16; Среднеохтинский п...",Памятник,POINT (3385859.012 8389439.579)
4,Особняк П.И. Иванова,"Большая Пороховская ул., 18",Памятник,POINT (3385918.345 8389472.261)


In [19]:
# переименовали столбцы object_name, object_location, object_type в русские названия
gdf_kgiop_objects = gdf_kgiop_objects.rename(columns ={'object_name': 'Объект культурного наследия', 'object_location': 'Адрес', 'object_type': 'Тип'})
gdf_kgiop_objects.head()

,Объект культурного наследия,Адрес,Тип,geometry
0,Казармы служителей Охтинского Порохового завод...,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",Ансамбль,POINT (3393917.541 8391682.542)
1,Казарма,"2-я Жерновская ул., 58, лит. А",Памятник,POINT (3393942.031 8391697.888)
2,Казарма,"2-я Жерновская ул., 64, лит. А",Памятник,POINT (3393893.051 8391664.084)
3,Доходный дом П.И. Иванова,"Большая Пороховская ул., 16; Среднеохтинский п...",Памятник,POINT (3385859.012 8389439.579)
4,Особняк П.И. Иванова,"Большая Пороховская ул., 18",Памятник,POINT (3385918.345 8389472.261)


In [20]:
gdf_kgiop_objects.explore(tiles=TILES)

### 4.3 Количество объектов культурного наследия, попадающее в буффер улицы

In [21]:
# создаем буфер
STREET_BUFFER = 100

def get_contains_kgiop_objects(street) -> int:
    """ Функция для подсчета количества объектов культурного наследия попадающих в буффер улицы"""
    return sum(gdf_kgiop_objects["geometry"].within(street.buffer(STREET_BUFFER)))


gdf_streets["contains_kgiop_objects"] = gdf_streets["geometry"].apply(get_contains_kgiop_objects)
gdf_streets.nlargest(5, "contains_kgiop_objects")

,geometry,contains_kgiop_objects
Название улицы,,
Большеохтинский проспект,"MULTILINESTRING ((3385140.255 8387624.984, 338...",6
Малоохтинский проспект,"MULTILINESTRING ((3384040.196 8383835.267, 338...",5
Новочеркасский проспект,"MULTILINESTRING ((3385451.037 8383969.096, 338...",5
Уральская дорожка,"MULTILINESTRING ((3387289.923 8388192.114, 338...",5
шоссе Революции,"MULTILINESTRING ((3392118.529 8391880.655, 339...",5


In [22]:
gdf_streets.explore("contains_kgiop_objects", tiles=TILES)

In [23]:
gdf_streets["density"] = gdf_streets["contains_kgiop_objects"] / gdf_streets.length

select_columns = []
gdf_streets.nlargest(5, "density")

,geometry,contains_kgiop_objects,density
Название улицы,,,
Большой Ильинский мост,"MULTILINESTRING ((3391992.838 8391901.138, 339...",1,0.005640
Конторская улица,"LINESTRING (3385162.853 8387874.512, 3385175.1...",3,0.005638
Уральская дорожка,"MULTILINESTRING ((3387289.923 8388192.114, 338...",5,0.004645
улица Помяловского,"MULTILINESTRING ((3385033.088 8385957.390, 338...",2,0.004028
Берёзовая дорожка,"LINESTRING (3386495.781 8388357.655, 3386630.5...",3,0.003725


In [24]:
# переименовать столбцы contains_kgiop_objects и density в русские названия
gdf_streets = gdf_streets.rename(columns ={'contains_kgiop_objects': 'Количество ОКН', 'density': 'Плотность'})

In [25]:
# уберем улицы, где нет объектов культурного наследия
gdf_streets = gdf_streets.loc[gdf_streets.loc[:,'Количество ОКН']>0]  # gdf_streets.loc[gdf_streets['Количество ОКН'] > 0]
gdf_streets.head(20)

,geometry,Количество ОКН,Плотность
Название улицы,,,
2-я Жерновская улица,"MULTILINESTRING ((3393793.008 8391638.241, 339...",3,0.001787
Берёзовая дорожка,"LINESTRING (3386495.781 8388357.655, 3386630.5...",3,0.003725
Большая Пороховская улица,"MULTILINESTRING ((3388222.992 8389322.413, 338...",3,0.000387
Большеохтинский проспект,"MULTILINESTRING ((3385140.255 8387624.984, 338...",6,0.001444
Большой Ильинский мост,"MULTILINESTRING ((3391992.838 8391901.138, 339...",1,0.005640
Ильинская слобода,"MULTILINESTRING ((3392005.450 8391883.835, 339...",4,0.003543
КАД,"MULTILINESTRING ((3394881.724 8396149.223, 339...",1,0.000026
Ковалёвская улица,"MULTILINESTRING ((3395555.774 8394603.666, 339...",1,0.000354
Конная дорога,"LINESTRING (3387262.149 8388623.636, 3387333.6...",2,0.002156


In [26]:
# выведем красивую карту с градацией объектов
m = gdf_streets.explore("Количество ОКН", tiles=TILES)
m = territory.explore(m=m, style_kwds={"fill": False, "weight": 5})

m

## 5 Выводы

In [27]:
total_kgiop_objects = gdf_streets['Количество ОКН'].sum()
print(f'Общее количество объектов культурного наследия на территории {TERRITORY_NAME} равно {total_kgiop_objects}.')

gdf_streets_kgiop_max = gdf_streets.nlargest(5, "Количество ОКН")
street_kgiop_max = gdf_streets_kgiop_max.index[0] #Самое большое количество объектов у улицы  или street_kgiop_max = gdf_streets_kgiop.indmax()
print(f'Самое большое количесво объектов у улицы - {street_kgiop_max}.')

top_5 = []
for address in gdf_streets.head().index:
    top_5.append(address)  # TODO list mprehenshion
top_5 = ', '.join(top_5)  # ', '.join(gdf_streets_kgiop_max.index)
print(f'Топ 5 улиц: {top_5}.')

density_avg = gdf_streets['Плотность'].mean()
print(f'Среднее количество объектов культурного наследия на одну улицу равно {density_avg.round(5)}.')

Общее количество объектов культурного наследия на территории Красногвардейский район, Санкт-Петербург равно 72.
Самое большое количесво объектов у улицы - Большеохтинский проспект.
Топ 5 улиц: 2-я Жерновская улица, Берёзовая дорожка, Большая Пороховская улица, Большеохтинский проспект, Большой Ильинский мост.
Среднее количество объектов культурного наследия на одну улицу равно 0.0014.


## 6 Сохранение файлов в формате geojson



In [28]:
gdf_kgiop_objects.to_crs(4236, inplace = True)

gdf_kgiop_objects['lon'] = gdf_kgiop_objects.geometry.x
gdf_kgiop_objects['lat'] = gdf_kgiop_objects.geometry.y

gdf_kgiop_objects.to_file('kgiop.geojson', driver='GeoJSON')

In [29]:
gdf_streets.to_crs(4236, inplace = True)

gdf_streets['lon'] = gdf_kgiop_objects.geometry.x
gdf_streets['lat'] = gdf_kgiop_objects.geometry.y

gdf_streets.to_file('streets_with_contains_kgiop.geojson', driver='GeoJSON')